In [1]:
import numpy as np, pandas as pd
import glob
import os 
import json
import sys
import numpy
from math import log, log10
from gdpyc import GasMap, DustMap
from astropy.coordinates import SkyCoord
from astropy import units as u

df = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_11152023_GammaCan.csv')

df = df[df['FGL'].str[-1]!='c']

df['GLON'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.l.degree
df['GLAT'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.b.degree

df['HR_ms'] = (df['Fcsc_m']-df['Fcsc_s'])/(df['Fcsc_m']+df['Fcsc_s'])
df['HR_hm'] = (df['Fcsc_h']-df['Fcsc_m'])/(df['Fcsc_h']+df['Fcsc_m'])
df = df.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df['name_nospace'] = df['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))


df_alls = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/Classifications/data/Individual_sources/FGL_11152023_GammaCan_alls.csv')

# light curve >50 counts
# spectrum > 100 counts
df_select3 = df 

# df_select3 = df[df['X_name'].isin(df_alls.loc[df_alls['src_cnts_aper90_b']>=50, 'name'].unique())].reset_index(drop=True)

# df_select3 = df_select3[~df_select3['X_name'].isin(['2CXO J174457.8-290509', '2CXO J174507.0-290357','2CXO J174506.8-290537','2CXO J174506.0-290512'])].reset_index(drop=True)
# print(df_select3)
# ,'2CXO J111459.1-611707','2CXO J150234.5+015205','2CXO J073717.0+653557'
coords = SkyCoord(df_select3['ra'], df_select3['dec'], unit='deg')
df_select3['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_select3['nH_from_AV'] = 2.21 * 3.1 * df_select3['ebv'] * 0.1
df_select3['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

print(df_select3[['FGL','X_name','Class','CT','name','name_nospace','ra', 'dec','flux_aper90_ave_b','var_intra_prob',
       'var_inter_prob','nH']])

              FGL                 X_name    Class        CT  \
0    J0859.3-4342  2CXO J085927.0-434528       CV  2.451807   
1    J1725.1-3408  2CXO J172508.8-341112      YSO  0.335596   
2    J0859.2-4729  2CXO J085905.6-473041      YSO  4.286869   
3    J1742.5-2833  2CXO J174216.9-283707       CV  1.907402   
4    J1743.8-3143  2CXO J174347.2-314025  LM-STAR  4.563578   
..            ...                    ...      ...       ...   
125  J1744.9-2905  2CXO J174506.8-290537       NS  2.130658   
126  J1744.9-2905  2CXO J174506.0-290512       NS  2.907012   
127  J1115.1-6118  2CXO J111513.6-611657       NS  4.325547   
128  J1115.1-6118  2CXO J111501.7-611751       CV  2.262834   
129  J1115.1-6118  2CXO J111459.1-611707       NS  2.373047   

                        name          name_nospace          ra        dec  \
0    2CXO J085927.0-434528-1  2CXOJ085927.0-434528  134.862609 -43.757805   
1    2CXO J172508.8-341112-1  2CXOJ172508.8-341112  261.286915 -34.186814   
2    2CXO J0

In [2]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ174457.8-290509':
    #     continue
    #print(src)
    # if index >40:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    try:
        dir_abs = f'/Users/huiyang/Research/GitHub/FGL_spectral_summary/bxa_results/{src}/'

        
        # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

        prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
        models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

        best = max(models, key=models.__getitem__)
        Zbest = models[best]
        for m in models: models[m] -= Zbest
        Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
        limit = 30 # for example, Jeffreys scale for the Bayes factor

        # print()
        # print('Model comparison')
        # print('****************')
        # print()



        for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
            df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
            # print(df_src_sum.columns)
            src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
            Zrel = models[m]
            src_stat['log10Z_'+md] = Zrel / log(10)
            if Zrel == 0:
                src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
        df_sr = pd.DataFrame(data=src_stat, index=[0])
        df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
        # for md in ['powerlaw','mekal','bb']:

        print('  <tr>')
        print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
        print(f"   <td>{src_stat['FGL']}</td>")
        print(f"   <td>{src_stat['l']:.7f}</td>")
        print(f"   <td>{src_stat['b']:.7f}</td>")
        print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        print(f"   <td>{src_stat['CT']:.2f}</td>")
        print(f"   <td>{src_stat['fb']:.2e}</td>")
        print(f"   <td>{src_stat['var_inter']:.2f}</td>")
        print(f'   <td><a class="image-link" href="./images/{src}_lc.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
        print(f"   <td>{src_stat['nobs']:.1f}</td>")
        print(f"   <td>{src_stat['ncounts']:.1f}</td>")
        print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}+{src_stat["pl_u"]-src_stat["pl"]:.2f}-{src_stat["pl"]-src_stat["pl_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}<sup>+{src_stat["pl_u"]-src_stat["pl"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{src_stat["pl"]-src_stat["pl_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}<sup>+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}<sup>+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a></td>')
        print(f"   <td>{src_stat['nH']:.2f}+{src_stat['nH_u']-src_stat['nH']:.2f}-{src_stat['nH']-src_stat['nH_l']:.2}</td>")
        # print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
        print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
        print('  </tr>')

    except:
        print('  <tr>')
        print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
        print(f"   <td>{src_stat['FGL']}</td>")
        print(f"   <td>{src_stat['l']:.7f}</td>")
        print(f"   <td>{src_stat['b']:.7f}</td>")
        print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        print(f"   <td>{src_stat['CT']:.2f}</td>")
        print(f"   <td>{src_stat['fb']:.2e}</td>")
        print(f"   <td>{src_stat['var_inter']:.2f}</td>")
        print(f'   <td>{src_stat["var_intra"]:.2f}</td>')
        print(f"   <td>{src_stat['nobs']:.1f}</td>")
        print(f"   <td>{src_stat['ncounts']:.1f}</td>")
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
        print('  </tr>')

    

  <tr>
   <td><a href='./srcs/2CXOJ085927.0-434528.html'>J085927.0-434528</a></td>
   <td>J0859.3-4342</td>
   <td>265.1473173</td>
   <td>1.4499867</td>
   <td><a class="image-link" href="./images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>
   <td>2.45</td>
   <td>1.59e-12</td>
   <td>nan</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_lc.png" target="_blank">1.00</a></td>
   <td>1.0</td>
   <td>6082.5</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_spectrum.png" target="_blank">1.63+0.06-0.054</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_corner.pdf" target="_blank">-10.45</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_spectrum.png" target="_blank">7.82+0.81-0.66</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_corner.pdf" target="_blank">0.00</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_bb_spectrum.png" tar

In [5]:
df_fgl_CSC_select= pd.read_csv(f'/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_11152023_class_perFGL_select.csv')
df_class = pd.read_csv(f'/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_11152023_class.csv')


class_labels = ['NS','HMXB','LMXB','AGN','CV','YSO','HM-STAR','LM-STAR']
df_class_select = df_class[df_class['FGL'].isin(df_fgl_CSC_select['FGL'])].reset_index(drop=True)
print(len(df_class_select),df_fgl_CSC_select['N_CSC'].sum())

# df_class_select = df_class_select[df_class_select['FGL'].str[-1]!='c'].reset_index(drop=True)
# df_class_select = df_class_select[df_class_select['FGL'].str[-1]=='c'].reset_index(drop=True)

df_class_select['significance'].describe()

1908 1902


count    1908.000000
mean        5.964036
std         4.661636
min         3.000000
25%         3.636364
50%         4.540541
75%         6.512331
max        73.143921
Name: significance, dtype: float64

In [20]:
df_class_select['sel'] = 'F'
df_class_select.loc[df_class_select['X_name'].isin(df_select3['X_name']), 'sel'] = 'Y'
print(df_class_select['sel'].value_counts())

df_class_select['GLON'] = SkyCoord(ra=df_class_select['ra'].values*u.degree, dec=df_class_select['dec'].values*u.degree, frame='icrs').galactic.l.degree
df_class_select['GLAT'] = SkyCoord(ra=df_class_select['ra'].values*u.degree, dec=df_class_select['dec'].values*u.degree, frame='icrs').galactic.b.degree

df_class_select['HR_ms'] = (df_class_select['Fcsc_m']-df_class_select['Fcsc_s'])/(df_class_select['Fcsc_m']+df_class_select['Fcsc_s'])
df_class_select['HR_hm'] = (df_class_select['Fcsc_h']-df_class_select['Fcsc_m'])/(df_class_select['Fcsc_h']+df_class_select['Fcsc_m'])
# df_class_select = df_class_select.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df_class_select[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df_class_select['name_nospace'] = df_class_select['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))

coords = SkyCoord(df_class_select['ra'], df_class_select['dec'], unit='deg')
df_class_select['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_class_select['nH_from_AV'] = 2.21 * 3.1 * df_class_select['ebv'] * 0.1
df_class_select['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

df_FGL = pd.read_csv(f'/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_11152023_class_perFGL.csv')


F    1778
Y     130
Name: sel, dtype: int64


In [21]:
print(len(df_class_select))
df_class_select.columns
df_class_select = df_class_select.sort_values(by=['FGL','X_name']).reset_index(drop=True)
for idx, row in df_class_select.iterrows():
    # print(row)
    print('        <tr>')
    print(f"            <td>{row['sel']}</td>")
    print(f"            <td>{row['FGL']}</td>")
    print(f"            <td>{row['X_name']}</td>")
    print(f"            <td>{row['significance']:.2f}</td>")
    print(f"            <td>{row['flux_aper90_ave_b']:.2e}</td>")
    print(f"            <td>{row['Class']}</td>")
    print(f"            <td>{row['CT']:.1f}</td>")
    # print(f"            <td>{row['']}</td>")
    # print(f"            <td>{row['']}</td>")
    print('        </tr>')

df_class_select.columns

1908
        <tr>
            <td>Y</td>
            <td>J0058.3-4603</td>
            <td>2CXO J005806.2-460419</td>
            <td>7.49</td>
            <td>2.27e-13</td>
            <td>AGN</td>
            <td>0.5</td>
        </tr>
        <tr>
            <td>Y</td>
            <td>J0335.6-0727</td>
            <td>2CXO J033532.7-072741</td>
            <td>8.21</td>
            <td>1.11e-13</td>
            <td>AGN</td>
            <td>2.3</td>
        </tr>
        <tr>
            <td>F</td>
            <td>J0442.8+3609</td>
            <td>2CXO J044240.2+360632</td>
            <td>4.00</td>
            <td>4.63e-15</td>
            <td>NS</td>
            <td>2.0</td>
        </tr>
        <tr>
            <td>F</td>
            <td>J0442.8+3609</td>
            <td>2CXO J044243.7+361008</td>
            <td>3.41</td>
            <td>9.15e-15</td>
            <td>AGN</td>
            <td>1.9</td>
        </tr>
        <tr>
            <td>F</td>
            <td>J0442.8+3609

Index(['P_AGN', 'P_CV', 'P_HM-STAR', 'P_HMXB', 'P_LM-STAR', 'P_LMXB', 'P_NS',
       'P_YSO', 'e_P_AGN', 'e_P_CV', 'e_P_HM-STAR', 'e_P_HMXB', 'e_P_LM-STAR',
       'e_P_LMXB', 'e_P_NS', 'e_P_YSO', 'Class', 'Class_prob', 'Class_prob_e',
       'name', 'CT', 'X_name', 'FGL', 'ra', 'dec', 'r0', 'significance',
       'Fcsc_s', 'e_Fcsc_s', 'Fcsc_m', 'e_Fcsc_m', 'Fcsc_h', 'e_Fcsc_h',
       'flux_aper90_ave_b', 'e_flux_aper90_ave_b', 'var_intra_prob',
       'var_inter_prob', 'MW_MW', 'MW_counts', 'match_flag', 'p_any', 'p_i',
       'Gmag', 'BPmag', 'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag', 'RPlx', 'PM',
       'epsi', 'sepsi', 'ruwe', 'rgeo', 'Jmag', 'Hmag', 'Kmag', 'e_Jmag',
       'e_Hmag', 'e_Kmag', 'W1mag', 'W2mag', 'W3mag', 'e_W1mag', 'e_W2mag',
       'e_W3mag', 'r_hat', 'X_PA', 'G_ra', 'G_dec', 'G_aF', 'G_bF', 'G_PA',
       'sep', 'Fb', 'HR_hms', 'popup', 'shape', 'color', 'sel', 'GLON', 'GLAT',
       'HR_ms', 'HR_hm', 'name_nospace', 'ebv', 'nH_from_AV', 'nH'],
      dtype='objec

In [22]:
df_class_select.columns

Index(['P_AGN', 'P_CV', 'P_HM-STAR', 'P_HMXB', 'P_LM-STAR', 'P_LMXB', 'P_NS',
       'P_YSO', 'e_P_AGN', 'e_P_CV', 'e_P_HM-STAR', 'e_P_HMXB', 'e_P_LM-STAR',
       'e_P_LMXB', 'e_P_NS', 'e_P_YSO', 'Class', 'Class_prob', 'Class_prob_e',
       'name', 'CT', 'X_name', 'FGL', 'ra', 'dec', 'r0', 'significance',
       'Fcsc_s', 'e_Fcsc_s', 'Fcsc_m', 'e_Fcsc_m', 'Fcsc_h', 'e_Fcsc_h',
       'flux_aper90_ave_b', 'e_flux_aper90_ave_b', 'var_intra_prob',
       'var_inter_prob', 'MW_MW', 'MW_counts', 'match_flag', 'p_any', 'p_i',
       'Gmag', 'BPmag', 'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag', 'RPlx', 'PM',
       'epsi', 'sepsi', 'ruwe', 'rgeo', 'Jmag', 'Hmag', 'Kmag', 'e_Jmag',
       'e_Hmag', 'e_Kmag', 'W1mag', 'W2mag', 'W3mag', 'e_W1mag', 'e_W2mag',
       'e_W3mag', 'r_hat', 'X_PA', 'G_ra', 'G_dec', 'G_aF', 'G_bF', 'G_PA',
       'sep', 'Fb', 'HR_hms', 'popup', 'shape', 'color', 'sel', 'GLON', 'GLAT',
       'HR_ms', 'HR_hm', 'name_nospace', 'ebv', 'nH_from_AV', 'nH'],
      dtype='objec

In [24]:
df_srcs = pd.DataFrame()


for index, df_src in df_class_select.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    fgl_ra, fgl_dec = df_FGL.loc[df_FGL['FGL']==df_src['FGL'], 'RAJ2000'].values[0], df_FGL.loc[df_FGL['FGL']==df_src['FGL'], 'DEJ2000'].values[0]

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    try:
        dir_abs = f'/Users/huiyang/Research/GitHub/FGL_spectral_summary/bxa_results/{src}/'

        
        # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

        prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
        models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

        best = max(models, key=models.__getitem__)
        Zbest = models[best]
        for m in models: models[m] -= Zbest
        Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
        limit = 30 # for example, Jeffreys scale for the Bayes factor

        # print()
        # print('Model comparison')
        # print('****************')
        # print()



        for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
            df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
            # print(df_src_sum.columns)
            src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
            Zrel = models[m]
            src_stat['log10Z_'+md] = Zrel / log(10)
            if Zrel == 0:
                src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
        df_sr = pd.DataFrame(data=src_stat, index=[0])
        df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
        # for md in ['powerlaw','mekal','bb']:

        print('  <tr>')

        print(f"   <td>{df_src['sel']}</td>")
        print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
        print(f"   <td>{src_stat['FGL']}</td>")
        print(f"   <td>{fgl_ra} {fgl_dec}</td>")
        print(f"   <td>{src_stat['l']:.7f}</td>")
        print(f"   <td>{src_stat['b']:.7f}</td>")
        print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        print(f"   <td>{src_stat['CT']:.2f}</td>")
        print(f"   <td>{df_src['significance']:.2f}</td>")
        print(f"   <td>{src_stat['fb']:.2e}</td>")
        print(f"   <td>{df_src['Fcsc_s']:.2e}</td>")
        print(f"   <td>{df_src['Fcsc_m']:.2e}</td>")
        print(f"   <td>{df_src['Fcsc_h']:.2e}</td>")
        print(f"   <td>{df_src['HR_hms']:.2f}</td>")
        print(f"   <td>{df_src['HR_ms']:.2f}</td>")
        print(f"   <td>{df_src['HR_hm']:.2f}</td>")
        print(f"   <td>{src_stat['var_inter']:.2f}</td>")
        print(f'   <td><a class="image-link" href="./images/{src}_lc.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
        print(f"   <td>{src_stat['nobs']:.1f}</td>")
        print(f"   <td>{src_stat['ncounts']:.1f}</td>")
        print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}+{src_stat["pl_u"]-src_stat["pl"]:.2f}-{src_stat["pl"]-src_stat["pl_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}<sup>+{src_stat["pl_u"]-src_stat["pl"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{src_stat["pl"]-src_stat["pl_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}<sup>+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</a></td>')
        # print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}<sup>+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</sub></a></td>')
        print(f'   <td><a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a></td>')
        print(f"   <td>{src_stat['nH']:.2f}+{src_stat['nH_u']-src_stat['nH']:.2f}-{src_stat['nH']-src_stat['nH_l']:.2}</td>")
        # print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
        print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
        print(f"   <td>{df_src['Gmag']:.2f}</td>")
        print(f"   <td>{df_src['BPmag']:.2f}</td>")
        print(f"   <td>{df_src['RPmag']:.2f}</td>")
        print(f"   <td>{df_src['Jmag']:.2f}</td>")
        print(f"   <td>{df_src['Hmag']:.2f}</td>")
        print(f"   <td>{df_src['Jmag']:.2f}</td>")
        print(f"   <td>{df_src['W1mag']:.2f}</td>")
        print(f"   <td>{df_src['W2mag']:.2f}</td>")
        print(f"   <td>{df_src['rgeo']:.0f}</td>")
        print(f"   <td>{df_src['RPlx']:.1f}</td>")
        print(f"   <td>{df_src['PM']:.5f}</td>")
        print(f"   <td>{df_src['epsi']:.2f}</td>")
        print(f"   <td>{df_src['ruwe']:.2f}</td>")
        print('  </tr>')



    except:
        print('  <tr>')

        print(f"   <td>{df_src['sel']}</td>")
        print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
        print(f"   <td>{src_stat['FGL']}</td>")
        print(f"   <td>{fgl_ra} {fgl_dec}</td>")
        print(f"   <td>{src_stat['l']:.7f}</td>")
        print(f"   <td>{src_stat['b']:.7f}</td>")
        print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
        print(f"   <td>{src_stat['CT']:.2f}</td>")
        print(f"   <td>{df_src['significance']:.2f}</td>")
        print(f"   <td>{src_stat['fb']:.2e}</td>")
        print(f"   <td>{df_src['Fcsc_s']:.2e}</td>")
        print(f"   <td>{df_src['Fcsc_m']:.2e}</td>")
        print(f"   <td>{df_src['Fcsc_h']:.2e}</td>")
        print(f"   <td>{df_src['HR_hms']:.2f}</td>")
        print(f"   <td>{df_src['HR_ms']:.2f}</td>")
        print(f"   <td>{df_src['HR_hm']:.2f}</td>")
        print(f"   <td>{df_src['var_inter_prob']:.2f}</td>")
        print(f"   <td>{df_src['var_intra_prob']:.2f}</td>")
        print(f"   <td></td>")
        print(f"   <td></td>")
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')
        print(f'   <td></td>')        
        print(f"   <td></td>")
        print(f"   <td>{df_src['Gmag']:.2f}</td>")
        print(f"   <td>{df_src['BPmag']:.2f}</td>")
        print(f"   <td>{df_src['RPmag']:.2f}</td>")
        print(f"   <td>{df_src['Jmag']:.2f}</td>")
        print(f"   <td>{df_src['Hmag']:.2f}</td>")
        print(f"   <td>{df_src['Jmag']:.2f}</td>")
        print(f"   <td>{df_src['W1mag']:.2f}</td>")
        print(f"   <td>{df_src['W2mag']:.2f}</td>")
        print(f"   <td>{df_src['rgeo']:.0f}</td>")
        print(f"   <td>{df_src['RPlx']:.1f}</td>")
        print(f"   <td>{df_src['PM']:.5f}</td>")
        print(f"   <td>{df_src['epsi']:.2f}</td>")
        print(f"   <td>{df_src['ruwe']:.2f}</td>")
        print('  </tr>')

    

  <tr>
   <td>Y</td>
   <td><a href='./srcs/2CXOJ005806.2-460419.html'>J005806.2-460419</a></td>
   <td>J0058.3-4603</td>
   <td>14.5904 -46.056</td>
   <td>299.3772577</td>
   <td>-71.0102805</td>
   <td><a class="image-link" href="./images/shap/J005806.2-460419_AGN.png" target="_blank">AGN</a></td>
   <td>0.50</td>
   <td>7.49</td>
   <td>2.27e-13</td>
   <td>8.10e-14</td>
   <td>4.33e-14</td>
   <td>1.03e-13</td>
   <td>-0.09</td>
   <td>-0.30</td>
   <td>0.41</td>
   <td>nan</td>
   <td><a class="image-link" href="./images/2CXOJ005806.2-460419_lc.png" target="_blank">0.47</a></td>
   <td>1.0</td>
   <td>76.8</td>
   <td><a class="image-link" href="./images/2CXOJ005806.2-460419_pl_spectrum.png" target="_blank">2.96+0.59-0.52</a></td>
   <td><a class="image-link" href="./images/2CXOJ005806.2-460419_pl_corner.pdf" target="_blank">-2.45</a></td>
   <td><a class="image-link" href="./images/2CXOJ005806.2-460419_mk_spectrum.png" target="_blank">1.52+0.27-0.18</a></td>
   <td><a class="ima